In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv(r'/Users/t_velpac/mission/WorkingCopy/Features_V3.csv')
dataset.drop(columns = ['Z_Score_HelpfulVotes','Z_Score_Words',"Date"], inplace=True)
dataset.rename(index=str, columns={"Helpful Votes": "Helpful_Votes"}, inplace=True)
dataset = dataset[dataset.Helpful_Votes != 0]
dataset = dataset[['Stars','Words', 'Paragraphs','No.break tags','Percentage_Upper_Case','Percentage_Lower_Case','Avg_len_paragraph_per_review','Helpful_Votes']]

dataset.head()

,Stars,Words,Paragraphs,No.break tags,Percentage_Upper_Case,Percentage_Lower_Case,Avg_len_paragraph_per_review,Helpful_Votes
0,4,268,8,14,3,90,166.125000,145
1,1,130,2,2,2,94,352.500000,77
2,1,110,1,0,3,90,556.000000,77
3,5,110,3,4,3,92,188.666667,1559
4,1,39,1,0,3,96,216.000000,51


In [3]:
"""Separating the independant variables from the dependant variable which is "Helpful Votes" in this case"""
X = dataset.iloc[:,0:-1].values
y = dataset.iloc[:,-1].values

In [4]:
"""Splitting the data into training data and testing data"""

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=1)

"""Scaling the data using StandardScaler from sklearn package"""
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [5]:
# importing keras and other required functions
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [6]:
""" Building a 3 layer ANN with high number of nodes """

regressor = Sequential()
regressor.add(Dense(100, kernel_initializer = 'normal',activation = 'relu',input_dim = 7))
regressor.add(Dense(100, kernel_initializer = 'normal', activation = 'relu'))
regressor.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
45528/45528 [==============================] - 8s 176us/step - loss: 54476.0355
Epoch 2/100
45528/45528 [==============================] - 8s 172us/step - loss: 54427.2898
Epoch 3/100
45528/45528 [==============================] - 8s 176us/step - loss: 54394.4083
Epoch 4/100
45528/45528 [==============================] - 8s 173us/step - loss: 54383.6619
Epoch 5/100
45528/45528 [==============================] - 8s 185us/step - loss: 54412.1929
Epoch 6/100
45528/45528 [==============================] - 7s 161us/step - loss: 54355.1546
Epoch 7/100
45528/45528 [==============================] - 7s 161us/step - loss: 54408.0148
Epoch 8/100
45528/45528 [==============================] - 8s 168us/step - loss: 54355.3026
Epoch 9/100
45528/45528 [==============================] - 8s 167us/step - loss: 54373.4112
Epoch 10/100
45528/45528 [==========================

45528/45528 [==============================] - 8s 169us/step - loss: 54234.9700
Epoch 83/100
45528/45528 [==============================] - 7s 163us/step - loss: 54276.1850
Epoch 84/100
45528/45528 [==============================] - 7s 161us/step - loss: 54208.2813
Epoch 85/100
45528/45528 [==============================] - 7s 164us/step - loss: 54252.5719
Epoch 86/100
45528/45528 [==============================] - 8s 167us/step - loss: 54263.3612
Epoch 87/100
45528/45528 [==============================] - 8s 178us/step - loss: 54235.7758
Epoch 88/100
45528/45528 [==============================] - 8s 172us/step - loss: 54242.4878 0s - loss: 51440.4
Epoch 89/100
45528/45528 [==============================] - 8s 173us/step - loss: 54256.2265
Epoch 90/100
45528/45528 [==============================] - 8s 169us/step - loss: 54267.3970
Epoch 91/100
45528/45528 [==============================] - 8s 166us/step - loss: 54193.7237
Epoch 92/100
45528/45528 [==============================] - 7s 1

In [7]:
""" Predicting the values for Helpful Votes on the test data """
y_pred = regressor.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred = y_pred.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred}
y_compare = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare['Mean Squared Error'] = (np.diff(y_compare.values) ** 2)
y_compare['Mean Squared Error'] = np.mean(y_compare['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare['Root Mean Squared Error'] = y_compare['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare['Mean Abs. Error'] = np.mean(abs(y_compare['Actual Values'] - y_compare['Predicted Values']))

y_compare

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,13.609477,61814.481726,248.625183,24.800995
1,13,20.194054,61814.481726,248.625183,24.800995
2,1,3.960111,61814.481726,248.625183,24.800995
3,1,10.220310,61814.481726,248.625183,24.800995
4,1,2.452003,61814.481726,248.625183,24.800995
5,2,9.677523,61814.481726,248.625183,24.800995
6,4,4.676023,61814.481726,248.625183,24.800995
7,1,60.972664,61814.481726,248.625183,24.800995
8,1,3.666610,61814.481726,248.625183,24.800995
9,5,32.228012,61814.481726,248.625183,24.800995


In [8]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train = regressor.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train = y_pred_train.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train }
y_compare_train = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train['Mean Squared Error'] = (np.diff(y_compare_train.values) ** 2)
y_compare_train['Mean Squared Error'] = np.mean(y_compare_train['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train['Root Mean Squared Error'] = y_compare_train['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train['Mean Abs. Error'] = np.mean(abs(y_compare_train['Actual Values(Training)'] - y_compare_train['Predicted Values(Training)']))

y_compare_train

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,7.944981,54123.873636,232.645382,24.364316
1,1,6.059867,54123.873636,232.645382,24.364316
2,2,20.895681,54123.873636,232.645382,24.364316
3,1,9.665132,54123.873636,232.645382,24.364316
4,1,8.495812,54123.873636,232.645382,24.364316
5,12,12.778905,54123.873636,232.645382,24.364316
6,5,10.450909,54123.873636,232.645382,24.364316
7,2968,37.880611,54123.873636,232.645382,24.364316
8,9,9.282689,54123.873636,232.645382,24.364316
9,1,6.254261,54123.873636,232.645382,24.364316


In [9]:
"""


Now we will be training a different ANN with lower number of nodes

7-4-4-1

"""

'\n\n\nNow we will be training a different ANN with lower number of nodes\n\n7-4-4-1\n\n'

In [10]:
""" Building a 3 layer ANN with less number of nodes """

regressor_1 = Sequential()
regressor_1.add(Dense(4, kernel_initializer = 'normal',activation = 'relu',input_dim = 7))
regressor_1.add(Dense(4, kernel_initializer = 'normal', activation = 'relu'))
regressor_1.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_1.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_1.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
45528/45528 [==============================] - 8s 174us/step - loss: 54545.1592
Epoch 2/100
45528/45528 [==============================] - 8s 169us/step - loss: 54413.0820
Epoch 3/100
45528/45528 [==============================] - 8s 177us/step - loss: 54394.0330
Epoch 4/100
45528/45528 [==============================] - 9s 188us/step - loss: 54378.0094
Epoch 5/100
45528/45528 [==============================] - 8s 181us/step - loss: 54362.7779
Epoch 6/100
45528/45528 [==============================] - 8s 180us/step - loss: 54376.4038
Epoch 7/100
45528/45528 [==============================] - 8s 179us/step - loss: 54358.9339
Epoch 8/100
45528/45528 [==============================] - 8s 171us/step - loss: 54366.8394
Epoch 9/100
45528/45528 [==============================] - 8s 179us/step - loss: 54356.4424
Epoch 10/100
45528/45528 [==============================] - 8s 177us/step - loss: 54358.9941
Epoch 11/100
45528/45528 [==============================] - 8s 182us/step - los

45528/45528 [==============================] - 7s 163us/step - loss: 54285.3482
Epoch 90/100
45528/45528 [==============================] - 7s 164us/step - loss: 54303.8093
Epoch 91/100
45528/45528 [==============================] - 7s 161us/step - loss: 54296.8709
Epoch 92/100
45528/45528 [==============================] - 7s 162us/step - loss: 54299.5981
Epoch 93/100
45528/45528 [==============================] - 7s 163us/step - loss: 54305.1690
Epoch 94/100
45528/45528 [==============================] - 7s 164us/step - loss: 54293.2500
Epoch 95/100
45528/45528 [==============================] - 7s 163us/step - loss: 54300.3153
Epoch 96/100
45528/45528 [==============================] - 7s 162us/step - loss: 54314.9660
Epoch 97/100
45528/45528 [==============================] - 7s 164us/step - loss: 54297.4116
Epoch 98/100
45528/45528 [==============================] - 7s 162us/step - loss: 54310.1946
Epoch 99/100
45528/45528 [==============================] - 7s 163us/step - loss: 5

In [11]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_1 = regressor_1.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_1 = y_pred_1.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_1}
y_compare_1 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_1['Mean Squared Error'] = (np.diff(y_compare_1.values) ** 2)
y_compare_1['Mean Squared Error'] = np.mean(y_compare_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_1['Root Mean Squared Error'] = y_compare_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_1['Mean Abs. Error'] = np.mean(abs(y_compare_1['Actual Values'] - y_compare_1['Predicted Values']))

y_compare_1

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,6.348928,61800.014822,248.596088,20.839386
1,13,9.552327,61800.014822,248.596088,20.839386
2,1,2.104369,61800.014822,248.596088,20.839386
3,1,6.158757,61800.014822,248.596088,20.839386
4,1,2.847720,61800.014822,248.596088,20.839386
5,2,9.038468,61800.014822,248.596088,20.839386
6,4,3.593949,61800.014822,248.596088,20.839386
7,1,33.580460,61800.014822,248.596088,20.839386
8,1,4.522549,61800.014822,248.596088,20.839386
9,5,15.726938,61800.014822,248.596088,20.839386


In [12]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_1 = regressor_1.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_1 = y_pred_train_1.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_1 }
y_compare_train_1 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_1['Mean Squared Error'] = (np.diff(y_compare_train_1.values) ** 2)
y_compare_train_1['Mean Squared Error'] = np.mean(y_compare_train_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_1['Root Mean Squared Error'] = y_compare_train_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_1['Mean Abs. Error'] = np.mean(abs(y_compare_train_1['Actual Values(Training)'] - y_compare_train_1['Predicted Values(Training)']))

y_compare_train_1

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,3.882396,54288.370015,232.998648,20.392892
1,1,4.510449,54288.370015,232.998648,20.392892
2,2,14.537847,54288.370015,232.998648,20.392892
3,1,5.993912,54288.370015,232.998648,20.392892
4,1,6.626829,54288.370015,232.998648,20.392892
5,12,15.160419,54288.370015,232.998648,20.392892
6,5,14.106667,54288.370015,232.998648,20.392892
7,2968,36.504150,54288.370015,232.998648,20.392892
8,9,7.030590,54288.370015,232.998648,20.392892
9,1,4.869841,54288.370015,232.998648,20.392892


In [13]:
"""


Now, we will train an ANN with a very high number of nodes. And comparitively more layers.

7-50-100-200-100-50-20-1
"""

'\n\n\nNow, we will train an ANN with a very high number of nodes. And comparitively more layers.\n\n7-50-100-200-100-50-20-1\n'

In [14]:

from keras.layers.advanced_activations import LeakyReLU
regressor_2 = Sequential()
regressor_2.add(Dense(50, kernel_initializer = 'normal',input_dim = 7))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(200, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(50, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(20, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_2.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_2.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
45528/45528 [==============================] - 12s 262us/step - loss: 54585.5735
Epoch 2/100
45528/45528 [==============================] - 11s 253us/step - loss: 54546.5386
Epoch 3/100
45528/45528 [==============================] - 11s 252us/step - loss: 54551.9483
Epoch 4/100
45528/45528 [==============================] - 12s 255us/step - loss: 54495.3823
Epoch 5/100
45528/45528 [==============================] - 12s 253us/step - loss: 54475.5559
Epoch 6/100
45528/45528 [==============================] - 12s 255us/step - loss: 54501.0471
Epoch 7/100
45528/45528 [==============================] - 12s 253us/step - loss: 54481.7490
Epoch 8/100
45528/45528 [==============================] - 12s 255us/step - loss: 54487.2271
Epoch 9/100
45528/45528 [==============================] - 12s 254us/step - loss: 54504.5935
Epoch 10/100
45528/45528 [==============================] - 11s 251us/step - loss: 54491.5695
Epoch 11/100
45528/45528 [==============================] - 12s 255us

45528/45528 [==============================] - 12s 254us/step - loss: 15264874035.9092
Epoch 88/100
45528/45528 [==============================] - 11s 252us/step - loss: 54729.6642
Epoch 89/100
45528/45528 [==============================] - 12s 257us/step - loss: 54263.7460
Epoch 90/100
45528/45528 [==============================] - 12s 256us/step - loss: 54294.6636
Epoch 91/100
45528/45528 [==============================] - 12s 253us/step - loss: 54436.9779
Epoch 92/100
45528/45528 [==============================] - 12s 254us/step - loss: 858027932.1469
Epoch 93/100
45528/45528 [==============================] - 12s 254us/step - loss: 55378.4821
Epoch 94/100
45528/45528 [==============================] - 12s 254us/step - loss: 54382.7923
Epoch 95/100
45528/45528 [==============================] - 12s 253us/step - loss: 54315.0069
Epoch 96/100
45528/45528 [==============================] - 12s 255us/step - loss: 54432.7207
Epoch 97/100
45528/45528 [==============================] - 12s

In [15]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_2 = regressor_2.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_2 = y_pred_2.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_2}
y_compare_2 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_2['Mean Squared Error'] = (np.diff(y_compare_2.values) ** 2)
y_compare_2['Mean Squared Error'] = np.mean(y_compare_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_2['Root Mean Squared Error'] = y_compare_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_2['Mean Abs. Error'] = np.mean(abs(y_compare_2['Actual Values'] - y_compare_2['Predicted Values']))

y_compare_2

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,8.699383,61785.190057,248.566269,24.863879
1,13,22.628086,61785.190057,248.566269,24.863879
2,1,10.285770,61785.190057,248.566269,24.863879
3,1,7.286279,61785.190057,248.566269,24.863879
4,1,10.182445,61785.190057,248.566269,24.863879
5,2,8.741915,61785.190057,248.566269,24.863879
6,4,9.835043,61785.190057,248.566269,24.863879
7,1,20.070637,61785.190057,248.566269,24.863879
8,1,6.934649,61785.190057,248.566269,24.863879
9,5,29.513790,61785.190057,248.566269,24.863879


In [16]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_2 = regressor_2.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_2 = y_pred_train_2.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_2 }
y_compare_train_2 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_2['Mean Squared Error'] = (np.diff(y_compare_train_2.values) ** 2)
y_compare_train_2['Mean Squared Error'] = np.mean(y_compare_train_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_2['Root Mean Squared Error'] = y_compare_train_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_2['Mean Abs. Error'] = np.mean(abs(y_compare_train_2['Actual Values(Training)'] - y_compare_train_2['Predicted Values(Training)']))

y_compare_train_2

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,9.075644,54298.392562,233.020155,24.303591
1,1,7.870188,54298.392562,233.020155,24.303591
2,2,21.722204,54298.392562,233.020155,24.303591
3,1,7.621979,54298.392562,233.020155,24.303591
4,1,8.036152,54298.392562,233.020155,24.303591
5,12,14.289301,54298.392562,233.020155,24.303591
6,5,11.705212,54298.392562,233.020155,24.303591
7,2968,36.342197,54298.392562,233.020155,24.303591
8,9,8.324979,54298.392562,233.020155,24.303591
9,1,8.234754,54298.392562,233.020155,24.303591


In [17]:
""" Building a 3 layer ANN with 2 layers of 50 nodes """

regressor_3 = Sequential()
regressor_3.add(Dense(50, kernel_initializer = 'normal',activation = 'relu',input_dim = 7))
regressor_3.add(Dense(50, kernel_initializer = 'normal', activation = 'relu'))
regressor_3.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_3.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_3.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
45528/45528 [==============================] - 8s 175us/step - loss: 54440.7775
Epoch 2/100
45528/45528 [==============================] - 8s 168us/step - loss: 54381.0735
Epoch 3/100
45528/45528 [==============================] - 8s 169us/step - loss: 54395.7717
Epoch 4/100
45528/45528 [==============================] - 8s 167us/step - loss: 54358.2022
Epoch 5/100
45528/45528 [==============================] - 8s 168us/step - loss: 54376.5768
Epoch 6/100
45528/45528 [==============================] - 8s 170us/step - loss: 54325.9146
Epoch 7/100
45528/45528 [==============================] - 8s 167us/step - loss: 54365.1366
Epoch 8/100
45528/45528 [==============================] - 8s 169us/step - loss: 54331.3032
Epoch 9/100
45528/45528 [==============================] - 8s 168us/step - loss: 54332.8905 0s - l
Epoch 10/100
45528/45528 [==============================] - 8s 171us/step - loss: 54315.9263
Epoch 11/100
45528/45528 [==============================] - 8s 167us/ste

Epoch 89/100
45528/45528 [==============================] - 8s 170us/step - loss: 54144.5467
Epoch 90/100
45528/45528 [==============================] - 8s 177us/step - loss: 54130.6220
Epoch 91/100
45528/45528 [==============================] - 8s 182us/step - loss: 54134.3918
Epoch 92/100
45528/45528 [==============================] - 8s 173us/step - loss: 54108.3298
Epoch 93/100
45528/45528 [==============================] - 8s 177us/step - loss: 54110.3219
Epoch 94/100
45528/45528 [==============================] - 8s 180us/step - loss: 54063.0987
Epoch 95/100
45528/45528 [==============================] - 8s 178us/step - loss: 54127.1378
Epoch 96/100
45528/45528 [==============================] - 8s 173us/step - loss: 54023.0962
Epoch 97/100
45528/45528 [==============================] - 9s 189us/step - loss: 54118.7264
Epoch 98/100
45528/45528 [==============================] - 9s 188us/step - loss: 54040.6293
Epoch 99/100
45528/45528 [==============================] - 8s 176us/s

In [18]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_3 = regressor_3.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_3 = y_pred_3.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_3}
y_compare_3 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_3['Mean Squared Error'] = (np.diff(y_compare_3.values) ** 2)
y_compare_3['Mean Squared Error'] = np.mean(y_compare_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_3['Root Mean Squared Error'] = y_compare_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_3['Mean Abs. Error'] = np.mean(abs(y_compare_3['Actual Values'] - y_compare_3['Predicted Values']))

y_compare_3

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,10.573049,61749.83855,248.495148,21.03294
1,13,17.723307,61749.83855,248.495148,21.03294
2,1,2.223211,61749.83855,248.495148,21.03294
3,1,5.417270,61749.83855,248.495148,21.03294
4,1,2.223211,61749.83855,248.495148,21.03294
5,2,6.512905,61749.83855,248.495148,21.03294
6,4,5.554589,61749.83855,248.495148,21.03294
7,1,49.886932,61749.83855,248.495148,21.03294
8,1,2.223211,61749.83855,248.495148,21.03294
9,5,26.243107,61749.83855,248.495148,21.03294


In [19]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_3 = regressor_3.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_3 = y_pred_train_3.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_3 }
y_compare_train_3 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_3['Mean Squared Error'] = (np.diff(y_compare_train_3.values) ** 2)
y_compare_train_3['Mean Squared Error'] = np.mean(y_compare_train_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_3['Root Mean Squared Error'] = y_compare_train_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_3['Mean Abs. Error'] = np.mean(abs(y_compare_train_3['Actual Values(Training)'] - y_compare_train_3['Predicted Values(Training)']))

y_compare_train_3

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,3.828350,54111.471346,232.618725,20.580289
1,1,5.383328,54111.471346,232.618725,20.580289
2,2,26.102842,54111.471346,232.618725,20.580289
3,1,7.330968,54111.471346,232.618725,20.580289
4,1,5.581612,54111.471346,232.618725,20.580289
5,12,10.459624,54111.471346,232.618725,20.580289
6,5,13.616932,54111.471346,232.618725,20.580289
7,2968,19.016562,54111.471346,232.618725,20.580289
8,9,4.745088,54111.471346,232.618725,20.580289
9,1,7.244819,54111.471346,232.618725,20.580289


In [20]:
""" Building a 3 layer ANN with 2 layers of 20 nodes """

regressor_4 = Sequential()
regressor_4.add(Dense(20, kernel_initializer = 'normal',activation = 'relu',input_dim = 7))
regressor_4.add(Dense(20, kernel_initializer = 'normal', activation = 'relu'))
regressor_4.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_4.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_4.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
45528/45528 [==============================] - 8s 186us/step - loss: 54506.1899
Epoch 2/100
45528/45528 [==============================] - 8s 177us/step - loss: 54378.1274
Epoch 3/100
45528/45528 [==============================] - 8s 181us/step - loss: 54376.5145
Epoch 4/100
45528/45528 [==============================] - 8s 178us/step - loss: 54361.6712
Epoch 5/100
45528/45528 [==============================] - 8s 183us/step - loss: 54363.2420
Epoch 6/100
45528/45528 [==============================] - 8s 180us/step - loss: 54330.3296
Epoch 7/100
45528/45528 [==============================] - 8s 178us/step - loss: 54332.7902
Epoch 8/100
45528/45528 [==============================] - 8s 183us/step - loss: 54324.5393
Epoch 9/100
45528/45528 [==============================] - 8s 177us/step - loss: 54326.6753
Epoch 10/100
45528/45528 [==============================] - 8s 179us/step - loss: 54332.1633
Epoch 11/100
45528/45528 [==============================] - 8s 181us/step - los

45528/45528 [==============================] - 8s 175us/step - loss: 54149.4077
Epoch 89/100
45528/45528 [==============================] - 8s 171us/step - loss: 54107.8124
Epoch 90/100
45528/45528 [==============================] - 8s 173us/step - loss: 54209.9253
Epoch 91/100
45528/45528 [==============================] - 8s 172us/step - loss: 54193.9691
Epoch 92/100
45528/45528 [==============================] - 8s 178us/step - loss: 54142.4321
Epoch 93/100
45528/45528 [==============================] - 8s 182us/step - loss: 54160.9203
Epoch 94/100
45528/45528 [==============================] - 8s 180us/step - loss: 54170.2042
Epoch 95/100
45528/45528 [==============================] - 8s 175us/step - loss: 54118.9518
Epoch 96/100
45528/45528 [==============================] - 8s 180us/step - loss: 54134.1202
Epoch 97/100
45528/45528 [==============================] - 8s 179us/step - loss: 54073.5871
Epoch 98/100
45528/45528 [==============================] - 8s 182us/step - loss: 5

In [21]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_4 = regressor_4.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_4 = y_pred_4.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_4}
y_compare_4 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_4['Mean Squared Error'] = (np.diff(y_compare_4.values) ** 2)
y_compare_4['Mean Squared Error'] = np.mean(y_compare_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_4['Root Mean Squared Error'] = y_compare_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_4['Mean Abs. Error'] = np.mean(abs(y_compare_4['Actual Values'] - y_compare_4['Predicted Values']))

y_compare_4

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,9.587395,61839.140147,248.674768,22.968661
1,13,20.155094,61839.140147,248.674768,22.968661
2,1,3.969570,61839.140147,248.674768,22.968661
3,1,7.699049,61839.140147,248.674768,22.968661
4,1,4.120923,61839.140147,248.674768,22.968661
5,2,4.446739,61839.140147,248.674768,22.968661
6,4,6.564935,61839.140147,248.674768,22.968661
7,1,51.905819,61839.140147,248.674768,22.968661
8,1,2.366548,61839.140147,248.674768,22.968661
9,5,32.566303,61839.140147,248.674768,22.968661


In [22]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_4 = regressor_4.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_4 = y_pred_train_4.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_4 }
y_compare_train_4 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_4['Mean Squared Error'] = (np.diff(y_compare_train_4.values) ** 2)
y_compare_train_4['Mean Squared Error'] = np.mean(y_compare_train_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_4['Root Mean Squared Error'] = y_compare_train_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_4['Mean Abs. Error'] = np.mean(abs(y_compare_train_4['Actual Values(Training)'] - y_compare_train_4['Predicted Values(Training)']))

y_compare_train_4

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,1,6.882870,54015.910827,232.413233,22.39806
1,1,4.777855,54015.910827,232.413233,22.39806
2,2,21.607674,54015.910827,232.413233,22.39806
3,1,8.304491,54015.910827,232.413233,22.39806
4,1,7.235771,54015.910827,232.413233,22.39806
5,12,14.272240,54015.910827,232.413233,22.39806
6,5,13.787373,54015.910827,232.413233,22.39806
7,2968,37.118534,54015.910827,232.413233,22.39806
8,9,4.882218,54015.910827,232.413233,22.39806
9,1,4.137470,54015.910827,232.413233,22.39806
